# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выберите данные по странам своей группы (совместно): <br>
    3530203_80101: Spain, Portugal, France, Italy, Belgium<br>
    3530203_80102: Germany, Netherlands <br>
    3530903_80301: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_80302: Sweden, Finland, Norway, Denmark, Iceland<br>
    
2. Попытайтесь найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Найдите и объясните хотя бы 5 правил.
3. Выведите эти правила в отдельных ячейках. 
4. Подумайте, как можно было бы использовать полученные правила на практике.

In [17]:
import pandas as pd

In [18]:
data_full = pd.read_csv("lastfm.csv")
data = data_full[
    (data_full['country'] == 'Spain') |
    (data_full['country'] == 'Portugal') |
    (data_full['country'] == 'France') |
    (data_full['country'] == 'Italy') |
    (data_full['country'] == 'Belgium')
]
data.info()
data.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27214 entries, 83 to 289762
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     27214 non-null  int64 
 1   artist   27214 non-null  object
 2   sex      27214 non-null  object
 3   country  27214 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


user                artist sex   country
83      6            lily allen   m  Portugal
84      6            kanye west   m  Portugal
85      6             sigur rós   m  Portugal
86      6            pink floyd   m  Portugal
87      6         stevie wonder   m  Portugal
88      6             metallica   m  Portugal
89      6  thievery corporation   m  Portugal
90      6           iron maiden   m  Portugal
91      6           the streets   m  Portugal
92      6                  muse   m  Portugal
93      6         faith no more   m  Portugal
94      6             manu chao   m  Portugal
95      6           tenacious d   m  Portugal
96      6          depeche mode   m  Portugal
97      6     justin timberlake   m  Portugal
98      6             green day   m  Portugal
99      6           snow patrol   m  Portugal
100     6         dream theater   m  Portugal
101     6                    u2   m  Portugal
102     6                 jay-z   m  Portugal

In [19]:
data.groupby(['user']).size().describe()

count    1419.000000
mean       19.178295
std        10.466411
min         1.000000
25%        11.000000
50%        19.000000
75%        27.000000
max        55.000000
dtype: float64

In [20]:
data_by_user = data.groupby(['user'])['artist'].apply(lambda x: '|'.join(x)).reset_index()
data_by_user

user                                             artist
0         6  lily allen|kanye west|sigur rós|pink floyd|ste...
1        12  queen|the beatles|lynyrd skynyrd|neil young|th...
2        36  m.i.a.|fugazi|johnny cash|clint mansell|elton ...
3        47  pj harvey|radiohead|zero 7|einstürzende neubau...
4        74  in extremo|nine inch nails|billy talent|panic!...
...     ...                                                ...
1414  19690  pink floyd|the beatles|the white stripes|depec...
1415  19693  air|simon & garfunkel|pixies|serge gainsbourg|...
1416  19697  underworld|pendulum|the prodigy|sufjan stevens...
1417  19704  keane|pinback|coldplay|travis|calexico|doves|d...
1418  19707  bat for lashes|mogwai|arcade fire|cat power|th...

[1419 rows x 2 columns]

In [21]:
dummy_data_by_user = data_by_user['artist'].str.get_dummies('|')
data_user = pd.concat([data_by_user, dummy_data_by_user], axis = 1)
data_user.sample(5)

user                                             artist  \
1174  16436  ben harper|tom waits|sufjan stevens|belle and ...   
186    2680  jack johnson|three days grace|good charlotte|s...   
60      762  amon tobin|dj krush|dj shadow|madlib|prefuse 7...   
948   13280  n*e*r*d|common|2pac|kanye west|arctic monkeys|...   
299    4152  coldplay|soundtrack|manu chao|the beatles|morc...   

      ...and you will know us by the trail of dead  2pac  3 doors down  \
1174                                             0     0             0   
186                                              0     0             1   
60                                               0     0             0   
948                                              0     1             0   
299                                              0     0             0   

      30 seconds to mars  311  36 crazyfists  44  50 cent  ...  wilco  \
1174                   0    0              0   0        0  ...      0   
186                    0    0              0   0        0  ...      0   
60                     0    0              0   0        0  ...      0   
948                    0    0              0   0        1  ...      0   
299                    0    0              0   0        0  ...      0   

      within temptation  wolfgang amadeus mozart  wu-tang clan  yann tiersen  \
1174                  0                        0             0             0   
186                   0                        0             0             0   
60                    0                        0             0             0   
948                   0                        0             0             0   
299                   0                        0             0             0   

      yeah yeah yeahs  yellowcard  yo la tengo  zero 7  Édith piaf  
1174                0           0            0       0           0  
186                 0           0            0       0           0  
60                  0           0            0       0           0  
948                 0           0            0       0           0  
299                 0           0            0       0           0  

[5 rows x 996 columns]

In [22]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules, fpmax

In [23]:
artist_sets = apriori(data_user.drop(columns=['user','artist']), 
                            min_support=0.03, use_colnames=True)
artist_sets

support                           itemsets
0    0.041579                        ([unknown])
1    0.043693                 (a perfect circle)
2    0.073996                            (ac/dc)
3    0.036646                        (aerosmith)
4    0.092319                              (air)
..        ...                                ...
264  0.035941                    (radiohead, u2)
265  0.035941  (the rolling stones, the beatles)
266  0.031712         (the beatles, the strokes)
267  0.039464         (the killers, the strokes)
268  0.038760        (muse, coldplay, radiohead)

[269 rows x 2 columns]

<h4>Какие группы чаще всего слушают вместе:</h4>

In [24]:
rules_conf = association_rules(artist_sets, metric="confidence", min_threshold=0.5)
rules_conf.sort_values('confidence', ascending=False).head(10)

antecedents        consequents  antecedent support  \
5            (keane)         (coldplay)            0.038055   
2             (beck)        (radiohead)            0.047216   
4             (blur)        (radiohead)            0.047921   
8      (snow patrol)         (coldplay)            0.050740   
16   (kaiser chiefs)      (the killers)            0.050740   
12   (kaiser chiefs)  (franz ferdinand)            0.050740   
7            (oasis)         (coldplay)            0.083157   
14        (interpol)        (radiohead)            0.079634   
21  (muse, coldplay)        (radiohead)            0.068358   
18       (sigur rós)        (radiohead)            0.086681   

    consequent support   support  confidence      lift  leverage  conviction  
5             0.188161  0.033122    0.870370  4.625676  0.025961    6.262760  
2             0.237491  0.032417    0.686567  2.890916  0.021204    2.432766  
4             0.237491  0.030303    0.632353  2.662637  0.018922    2.074024  
8             0.188161  0.031712    0.625000  3.321629  0.022165    2.164905  
16            0.109937  0.031008    0.611111  5.558761  0.025430    2.288735  
12            0.095842  0.030303    0.597222  6.231311  0.025440    2.244806  
7             0.188161  0.047921    0.576271  3.062655  0.032274    1.915941  
14            0.237491  0.045807    0.575221  2.422074  0.026895    1.795073  
21            0.237491  0.038760    0.567010  2.387500  0.022525    1.761032  
18            0.237491  0.047921    0.552846  2.327857  0.027335    1.705247

<h4>Способ определить зависимость между разными исполнителями:</h4>
lift(A→C) = confidence(A→C) / support(C), range: [0,∞]

In [25]:
rules_conf.sort_values('lift', ascending=False).head(10)

antecedents        consequents  antecedent support  \
12  (kaiser chiefs)  (franz ferdinand)            0.050740   
16  (kaiser chiefs)      (the killers)            0.050740   
5           (keane)         (coldplay)            0.038055   
15    (iron maiden)        (metallica)            0.073291   
8     (snow patrol)         (coldplay)            0.050740   
7           (oasis)         (coldplay)            0.083157   
13       (interpol)             (muse)            0.079634   
2            (beck)        (radiohead)            0.047216   
10             (u2)         (coldplay)            0.088090   
9     (the killers)         (coldplay)            0.109937   

    consequent support   support  confidence      lift  leverage  conviction  
12            0.095842  0.030303    0.597222  6.231311  0.025440    2.244806  
16            0.109937  0.031008    0.611111  5.558761  0.025430    2.288735  
5             0.188161  0.033122    0.870370  4.625676  0.025961    6.262760  
15            0.123326  0.039464    0.538462  4.366154  0.030426    1.899460  
8             0.188161  0.031712    0.625000  3.321629  0.022165    2.164905  
7             0.188161  0.047921    0.576271  3.062655  0.032274    1.915941  
13            0.174066  0.040874    0.513274  2.948730  0.027012    1.696918  
2             0.237491  0.032417    0.686567  2.890916  0.021204    2.432766  
10            0.188161  0.045102    0.512000  2.721079  0.028527    1.663605  
9             0.188161  0.055673    0.506410  2.691371  0.034987    1.644765

<h4>Другой способ определить зависимость между разными исполнителями:</h4>
levarage(A→C) = support(A→C) − support(A) × support(C), range: [−1,1]

In [26]:
rules_conf.sort_values('leverage', ascending=False).head(10)

antecedents  consequents  antecedent support  consequent support  \
9   (the killers)   (coldplay)            0.109937            0.188161   
7         (oasis)   (coldplay)            0.083157            0.188161   
15  (iron maiden)  (metallica)            0.073291            0.123326   
10           (u2)   (coldplay)            0.088090            0.188161   
18    (sigur rós)  (radiohead)            0.086681            0.237491   
13     (interpol)       (muse)            0.079634            0.174066   
14     (interpol)  (radiohead)            0.079634            0.237491   
0           (air)  (radiohead)            0.092319            0.237491   
5         (keane)   (coldplay)            0.038055            0.188161   
6          (moby)   (coldplay)            0.081043            0.188161   

     support  confidence      lift  leverage  conviction  
9   0.055673    0.506410  2.691371  0.034987    1.644765  
7   0.047921    0.576271  3.062655  0.032274    1.915941  
15  0.039464    0.538462  4.366154  0.030426    1.899460  
10  0.045102    0.512000  2.721079  0.028527    1.663605  
18  0.047921    0.552846  2.327857  0.027335    1.705247  
13  0.040874    0.513274  2.948730  0.027012    1.696918  
14  0.045807    0.575221  2.422074  0.026895    1.795073  
0   0.048626    0.526718  2.217840  0.026701    1.611107  
5   0.033122    0.870370  4.625676  0.025961    6.262760  
6   0.040874    0.504348  2.680410  0.025625    1.637921

<h4>Самая часто прослушиваемая группа и что слушают вместе с ней</h4>

In [27]:
rules_conf.sort_values(by=['consequent support', 'antecedent support'], ascending=[False, False]).head(10)

antecedents  consequents  antecedent support  \
0                   (air)  (radiohead)            0.092319   
17       (massive attack)  (radiohead)            0.088090   
19          (the strokes)  (radiohead)            0.087385   
18            (sigur rós)  (radiohead)            0.086681   
14             (interpol)  (radiohead)            0.079634   
3                 (björk)  (radiohead)            0.074700   
11  (death cab for cutie)  (radiohead)            0.069767   
21       (muse, coldplay)  (radiohead)            0.068358   
1           (arcade fire)  (radiohead)            0.064834   
20    (the white stripes)  (radiohead)            0.063425   

    consequent support   support  confidence      lift  leverage  conviction  
0             0.237491  0.048626    0.526718  2.217840  0.026701    1.611107  
17            0.237491  0.045102    0.512000  2.155869  0.024182    1.562518  
19            0.237491  0.045807    0.524194  2.207213  0.025054    1.602561  
18            0.237491  0.047921    0.552846  2.327857  0.027335    1.705247  
14            0.237491  0.045807    0.575221  2.422074  0.026895    1.795073  
3             0.237491  0.040874    0.547170  2.303958  0.023133    1.683874  
11            0.237491  0.035941    0.515152  2.169139  0.019372    1.572674  
21            0.237491  0.038760    0.567010  2.387500  0.022525    1.761032  
1             0.237491  0.033827    0.521739  2.196878  0.018429    1.594337  
20            0.237491  0.033122    0.522222  2.198912  0.018059    1.595949

<h4>Еще один способ определить зависимость между разными исполнителями</h4>
conviction(A→C) = (1 − support(C)) / (1 − confidence(A→C)), range: [0,∞]

In [28]:
rules_conf.sort_values('conviction', ascending=False).head(10)

antecedents        consequents  antecedent support  \
5            (keane)         (coldplay)            0.038055   
2             (beck)        (radiohead)            0.047216   
16   (kaiser chiefs)      (the killers)            0.050740   
12   (kaiser chiefs)  (franz ferdinand)            0.050740   
8      (snow patrol)         (coldplay)            0.050740   
4             (blur)        (radiohead)            0.047921   
7            (oasis)         (coldplay)            0.083157   
15     (iron maiden)        (metallica)            0.073291   
14        (interpol)        (radiohead)            0.079634   
21  (muse, coldplay)        (radiohead)            0.068358   

    consequent support   support  confidence      lift  leverage  conviction  
5             0.188161  0.033122    0.870370  4.625676  0.025961    6.262760  
2             0.237491  0.032417    0.686567  2.890916  0.021204    2.432766  
16            0.109937  0.031008    0.611111  5.558761  0.025430    2.288735  
12            0.095842  0.030303    0.597222  6.231311  0.025440    2.244806  
8             0.188161  0.031712    0.625000  3.321629  0.022165    2.164905  
4             0.237491  0.030303    0.632353  2.662637  0.018922    2.074024  
7             0.188161  0.047921    0.576271  3.062655  0.032274    1.915941  
15            0.123326  0.039464    0.538462  4.366154  0.030426    1.899460  
14            0.237491  0.045807    0.575221  2.422074  0.026895    1.795073  
21            0.237491  0.038760    0.567010  2.387500  0.022525    1.761032

In [29]:
artist_sets = fpgrowth(data_user.drop(columns=['user','artist']), 
                            min_support=0.017, use_colnames=True)
rules_lift = association_rules(artist_sets, metric="lift")

<h4>Те группы, которые редко слушают вместе, т.е. их не стоит рекомендовать</h4>

In [30]:
rules_lift[rules_lift['lift'] < 1].head(10)

antecedents         consequents  antecedent support  \
46           (metallica)         (radiohead)            0.123326   
47           (radiohead)         (metallica)            0.237491   
48           (metallica)          (coldplay)            0.123326   
49            (coldplay)         (metallica)            0.188161   
184       (led zeppelin)          (coldplay)            0.098661   
185           (coldplay)      (led zeppelin)            0.188161   
1108         (radiohead)  (system of a down)            0.237491   
1109  (system of a down)         (radiohead)            0.082452   
1576       (linkin park)         (radiohead)            0.083862   
1577         (radiohead)       (linkin park)            0.237491   

      consequent support   support  confidence      lift  leverage  conviction  
46              0.237491  0.024665    0.200000  0.842136 -0.004624    0.953136  
47              0.123326  0.024665    0.103858  0.842136 -0.004624    0.978275  
48              0.188161  0.021846    0.177143  0.941445 -0.001359    0.986610  
49              0.123326  0.021846    0.116105  0.941445 -0.001359    0.991830  
184             0.188161  0.017618    0.178571  0.949037 -0.000946    0.988326  
185             0.098661  0.017618    0.093633  0.949037 -0.000946    0.994453  
1108            0.082452  0.019027    0.080119  0.971696 -0.000554    0.997463  
1109            0.237491  0.019027    0.230769  0.971696 -0.000554    0.991261  
1576            0.237491  0.017618    0.210084  0.884597 -0.002298    0.965304  
1577            0.083862  0.017618    0.074184  0.884597 -0.002298    0.989547